In [24]:
import os
from audiolazy.lazy_midi import str2midi
from mido import Message, MetaMessage, MidiFile, MidiTrack
from mido import bpm2tempo, tempo2bpm, tick2second, second2tick

In [43]:
def chord_to_midi(chords_str, chord_len=4, ticks_per_beat=96):
    track = MidiTrack()
    track.append(MetaMessage('track_name', name='\x00', time=0))
    track.append(MetaMessage('time_signature', numerator=4, denominator=4, 
                             clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0))
    
    chords = chords_str.split(' ')
    for chord in chords:
        notes = chord.split('_')
        midi_notes = [str2midi(note.replace('-', 'b')) for note in notes]
        for midi_note in midi_notes:
            track.append(Message('note_on', channel=0, note=midi_note, velocity=100, time=0))
        for j, midi_note in enumerate(midi_notes):
            time = ticks_per_beat*chord_len if j == 0 else 0
            track.append(Message('note_off', channel=0, note=midi_note, velocity=100, time=time))
    
    track.append(MetaMessage('end_of_track', time=0))
    return track

In [44]:
def store_chords(track, filepath, ticks_per_beat=96):
    mid = MidiFile()
    mid.tracks.append(track)
    mid.ticks_per_beat = ticks_per_beat
    mid.filename = os.path.basename(filepath)
    mid.save(filepath)    
    return mid

In [45]:
tests = [
'E0_E2 E0_E2 B-1_E-3_C4_E-4_G4 E-0_G2_E-3',
]

for i, test in enumerate(tests):
    track = chord_to_midi(test)
    store_chords(track, f'seq{i+1}.mid')
